In [2]:
from kamene.all import *
from pynq_networking import *

A = 0.1 # Smoothing factor
WINDOW_SIZE = 30 # in seconds
OVERLAP = 0.8
INTERVAL_SEC = WINDOW_SIZE - WINDOW_SIZE*OVERLAP

# Observed entropies
H_src = 0
H_sport = 0
H_dport = 0

# Predicted entropies
H_src_predict = 0
H_sport_predict = 0
H_dport_predict = 0

d = 0         # Anomaly score
err_src = 0   # Source IP prediction error
err_sport = 0 # Source port prediction error
err_dport = 0 # Destination port prediction error

In [ ]:
overlap_flows = [[] for i in range(OVERLAP/(1-OVERLAP))]
current_flows = []
def shift_flows():
    for i in range(len(overlap_flows-1)):
        overlap_flows[i] = overlap_flows[i+1]
    overlap_flows[len(overlap_flows)-1] = current_flows

In [1]:
def packet_processing(packet):
    if(TCP in packet):
        print("Source: ", packet[TCP].src, ":", packet[TCP].sport,
             "\nDestination port: ", packet[TCP].dport)
        flow = {"src": packet[TCP].src, "sport":packet[TCP].sport, "dport": packet[TCP].dport}
        if flow not in current_flows:
            current_flows.append({flow: 0})
        else:
            current_flows[flow] = current_flows[flow] + 1
    elif(UDP in packet):
        print("Source: ", packet[UDP].src, ":", packet[UDP].sport,
             "\nDestination port: ", packet[UDP].dport)
        flow = {"src": packet[UDP].src, "sport":packet[UDP].sport, "dport": packet[UDP].dport}
        if flow not in current_flows:
            current_flows.append({flow: 0})
        else:
            current_flows[flow] = current_flows[flow] + 1

In [ ]:
def calc_entropy():
    src = []
    sport = []
    dport = []
    for x in overlap_flows:
        for flow in x:
            if(flow["src"] in src):
                src.append(flow["src"] : 0)
            else:
                src[flow["src"]] = src[flow["src"]] + 1
            
            if(flow["sport"] in src):
                src.append(flow["sport"] : 0)
            else:
                src[flow["sport"]] = src[flow["sport"]] + 1
                
            if(flow["dport"] in src):
                src.append(flow["dport"] : 0)
            else:
                src[flow["dport"]] = src[flow["dport"]] + 1
    
def predict_entropy():
    
    
def calc_error():

In [ ]:
import signal, time
from IPython.display import clear_output

def time_out(signum, frame):
    raise KeyboardInterrupt

for i in range(len(overlap_flows)):
    clear_output(wait=True)
    pkt_trace = sniff(iface="br0",
                      timeout = INTERVAL_SEC, filter="tcp or udp",
                      prn=lambda x: packet_processing(x))
    shift_flows()
    
pkt_trace = sniff(iface="br0",
                  timeout = INTERVAL_SEC, filter="tcp or udp",
                  prn=lambda x: packet_processing(x))
calc_entropy()
predict_entropy()

In [ ]:
while(True):
    clear_output(wait=True)
    pkt_trace = sniff(iface="br0",
                      timeout = INTERVAL_SEC, filter="tcp or udp",
                      prn=lambda x: packet_processing(x))
    calc_entropy()
    calc_error()
    predict_entropy()

In [ ]:
import matplotlib.pyplot as plt
from collections import Counter

def draw_graphs():
    while(True):
        types = ["ARP" if ARP in x else 
                 "MQTTSN" if MQTTSN in x else
                 "IPV6_UDP" if ((UDP in x) and (IPv6 in x)) else
                 "UDP" if (UDP in x) else
                 "IPV6_TCP" if ((TCP in x) and (IPv6 in x)) else
                 "TCP" if (TCP in x) else
                 "Unknown" for x in pkt_trace]
        counts = Counter(types)
        frequencies = counts.values()
        names = [x for x in counts.keys()]
        centers = range(len(counts))
        plt.bar(centers, frequencies, align='center', tick_label=names)
        plt.title("Histogram of received packet types")
        plt.draw()
        plt.pause(INTERVAL_SEC)
        plt.cla()